# sbpy.activity.dust

[sbpy.activity.dust](https://sbpy.readthedocs.io/en/latest/sbpy/activity.html) is used for modeling cometary dust comae.  Comet brightness can be estimated for observations of scattered light or themral emission.

## Light scattered by dust via Afρ

Light scattered by coma dust can be estimated via the cometary parameter Afρ.  For a circular aperture, the Αfρ quantity is the product of albedo, filling factor of dust in the aperture, and aperture radius.  It has units of length, is aperture-size independent for an idealized coma, and is proportional to mass loss rate under certain assumptions (see [A'Hearn et al. 1984 for details](https://ui.adsabs.harvard.edu/#abs/1984AJ.....89..579A/abstract)):

$$ Af\rho = \frac{(2 r_h \Delta)^2}{\rho}\frac{F_\lambda}{S_\lambda} $$

where $r_h$ is the heliocentric distance in au, $\Delta$ is the observer-comet distance in the same units as $\rho$, $F_\lambda$ is the flux desnity of the comet continuum, and $S_\lambda$ is the flux desnity of sunlight at 1 au in the same units as $F_\lambda$.

`sbpy` has a class that represents this parameter: `Afrho`.  With this quantity, we can estimate the brightness of a comet, under the assumptions of the Afρ model (isotropic coma in free expansion).

`Afrho` is an `astropy` `Quantity` that has units of length.

In [1]:
import astropy.units as u
from sbpy.activity import Afrho

afrho = Afrho(100 * u.cm)
afrho = Afrho('100 cm')  # alternate initialization
afrho = Afrho(100, unit='cm')  # alternate initialization

print(afrho)

100.0 cm


In [2]:
# Arithmetic works as usual:
print(afrho * 2)

200.0 cm


In [3]:
# unit conversion, too
print(afrho.to('m'))

1.0 m


`Afrho` can also be initialized from continuum flux densities.  Let's work with some photometry of comet 46P/Wirtanen by [Farnham et al. (1998)](https://ui.adsabs.harvard.edu/#abs/1998A&A...335L..50F/abstract).  The following observations were taken through the IHW blue continuum filter (λ=4845 Å).

In [4]:
import numpy as np

# comet ephemeris as a dictionary:
eph = {
    'rh': [1.12, 1.14, 1.07, 1.49, 1.72] * u.au,
    'delta': [1.41, 1.60, 1.54, 1.96, 2.31] * u.au,
    'phase': [45, 38, 40, 31, 24] * u.deg
}

# observational data:
wave = 4845 * u.AA
rho = 10**np.array((4.29, 4.64, 4.49, 5.03, 4.96)) * u.km
fluxd = 10**np.array((-13.74, -13.84, -13.48, -14.11, -14.30)) * u.erg / (u.cm**2 * u.s * u.AA)

afrho = Afrho.from_fluxd(wave, fluxd, rho, eph)
print(afrho)

[ 105.8130834    50.08581756  132.27388103   28.0898732    39.44082157] cm


Compare with the Farnham et al. values:

In [5]:
f98 = 10**np.array((2.05, 1.71, 2.14, 1.47, 1.61)) * u.cm
print('Mean percent difference: {:.1%}'.format(np.mean(np.abs(afrho - f98) / f98)))

Mean percent difference: 4.0%


The few percent difference is due to the assumed solar flux density.  We asked for the conversion at a specific wavelength, but the observation was through a filter with a specific width.  With the `spectroscopy` module we can show the solar flux density used:

In [6]:
from sbpy.spectroscopy.sun import default_sun
sun = default_sun.get()
print(sun(wave, unit=fluxd.unit))

196.89999999999878 erg / (Angstrom cm2 s)


But rather than having `sbpy` compute the solar flux density at 1 au, we can pass the precomputed value for this filter from [A'Hearn et al. 1995](https://ui.adsabs.harvard.edu/#abs/1995Icar..118..223A/abstract):

In [7]:
S = 189.7 * u.erg / (u.cm**2 * u.s * u.AA)
afrho = Afrho.from_fluxd(None, fluxd, rho, eph, S=S)
print('Mean percent difference: {:.1%}'.format(np.mean(np.abs(afrho - f98) / f98)))

Mean percent difference: 1.1%


In the future, we will be able to provide the filter transmission and have `sbpy` automatically compute the mean solar flux density in the filter.